In [1]:
%gui asyncio

In [2]:
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [3]:
from ipywidgets import IntSlider, Output
slider = IntSlider()
out = Output()

async def f():
    for i in range(10):
        out.append_stdout('did work ' + str(i) + '\n')
        x = await wait_for_change(slider, 'value')
        out.append_stdout('async function continued with value ' + str(x) + '\n')
asyncio.ensure_future(f())

slider

IntSlider(value=0)

In [4]:
from functools import wraps
def yield_for_change(widget, attribute):
    """Pause a generator to wait for a widget change event.

    This is a decorator for a generator function which pauses the generator on yield
    until the given widget attribute changes. The new value of the attribute is
    sent to the generator and is the value of the yield.
    """
    def f(iterator):
        @wraps(iterator)
        def inner():
            i = iterator()
            def next_i(change):
                try:
                    i.send(change.new)
                except StopIteration as e:
                    widget.unobserve(next_i, attribute)
            widget.observe(next_i, attribute)
            # start the generator
            next(i)
        return inner
    return f

In [10]:
from ipywidgets import IntSlider, VBox, HTML
slider2=IntSlider()

@yield_for_change(slider2, 'value')
def f():
    for i in range(10):
        print('did work %s'%i)
        x = yield
        print('generator function continued with value %s'%x)
f()





did work 0


In [11]:
import threading
from IPython.display import display
import ipywidgets as widgets
import time


FloatProgress(value=0.0, max=1.0)

In [12]:
slider2

IntSlider(value=0)

generator function continued with value 1
did work 1
generator function continued with value 2
did work 2
generator function continued with value 3
did work 3
generator function continued with value 4
did work 4
generator function continued with value 5
did work 5
generator function continued with value 6
did work 6
generator function continued with value 7
did work 7
generator function continued with value 8
did work 8
generator function continued with value 9
did work 9
generator function continued with value 10


In [18]:
class sim:
    def __init__(self):
        self.slider2=IntSlider()
        self.slider2.value = 1.0
        self.progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)

    def work(self,progress):
        total = 100
        for i in range(total):
            time.sleep(1/self.slider2.value)
            progress.value = float(i+1)/total

sim_obj = sim()
thread = threading.Thread(target=sim_obj.work, args=(sim_obj.progress,))
display(sim_obj.progress)
thread.start()
sim_obj.slider2


FloatProgress(value=0.0, max=1.0)

IntSlider(value=1)